# Tessellata: Getting Started

Tessellata is an aggregation and distribution system for oceanic data. It is a collaborative effort between Sparkgeo and WCS.

This guide is an example that requests and combines public data from Mermaid and Tessellata, and plots the output.

To execute a cell, press Shift + Enter.

## Import dependencies

In [ ]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
from ipyleaflet import Map, Marker, MarkerCluster, TileLayer

## Set up links to endpoints

In [ ]:
mermaid_api = "https://dev-api.datamermaid.org/v1"
titiler_endpoint = "https://9rhnl96o03.execute-api.us-east-1.amazonaws.com"
stac_endpoint = "http://stac/endpoint"
tessellata_api = "https://tessellata/endpoint"

## Create a basic function for making requests

In [ ]:
def send_request(api, endpoint, params):
    response = requests.get(f"{api}/{endpoint}", params=params)
    print(f"REQUEST URL: {response.request.path_url}")
    return response

## Create a list of Mermaid projects that have a public sharing policy

In [ ]:
policy = "data_policy_beltfish"

endpoint = "projects"

params = {
    "limit": "1000",
    "showall": True,
    "status": "90",
    policy: "50"
}
response = send_request(mermaid_api, endpoint, params)

results = response.json()["results"]
projects = []
for result in results:
    if result["num_sites"] > 0:
        projects.append(result["name"])
        
print(projects)

## Collect summary site data from Mermaid

In [ ]:
endpoint = "summarysites"

summary_results = []
for project in projects:
    params = {
        "page": 1,
        "limit": 1000,
        "project_name": project
    }
    response = send_request(mermaid_api, endpoint, params)
    response_json = response.json()

    if response_json["count"] > 0:
        summary_results.append(response_json["results"])

## Parse the results into a organized structure

In [ ]:
summary_stats = []
protocol = "beltfish"
variable = "biomass_kgha_avg"

for result in summary_results:
    for site in result:
        summary_stats.append([
            site["project_name"],
            site["site_name"],
            site["latitude"],
            site["longitude"],
            site["protocols"].get(protocol, {}).get(variable)
        ])
            
df = pd.DataFrame(summary_stats, columns=['project_name', 'site_name', 'latitude', 'longitude', 'value'])

In [ ]:
df

## Map the projects

In [ ]:
m = Map(center=(0,0), zoom=2)

lats = df['latitude']
lngs = df['longitude']
site_names = df['site_name']
sites = zip(list(lats), list(lngs), list(site_names))

markers = []
for site in sites:
    marker = Marker(
        location=(site[0], site[1]),
        title=site[2]
    )
    markers.append(marker)

layer_group = MarkerCluster(markers=markers)
m.add_layer(layer_group);

display(m)

## Plot the distribution of our data

In [ ]:
df['value'].plot.hist(bins=20)

## Set up mock data

In [ ]:
# mocked responses
import random
import numpy as np
import uuid

def make_mocked_response():
    mocked_arr = np.array(np.random.randint([100,100]))
    arr_min = np.min(mocked_arr)
    arr_max = np.max(mocked_arr)
    arr_mean = np.mean(mocked_arr)
    arr_median = np.median(mocked_arr)
    item_id = str(uuid.uuid4())
    collection_id = "fishing_pressure"
    
    return {
      "numberMatched": 20,
      "numberReturned": 1,
      "type": "FeatureCollection",
      "meta": {
        "next": f"{stac_endpoint}/stac/search?page=2&limit=2&datetime=2018-12-29T00:00:00Z/2019-01-01T12:31:12Z&bbox0=-110.039063&bbox1=35.029996&bbox2=-85.957031&bbox3=45.95115&collections=sentinel-s2-l2a-cogs",
        "previous": None,
        "count": 20,
        "limit": 2
      },
      "features": [
        {
          "stac_version": "1.0.0-beta.2",
          "stac_extensions": ["eo", "view", "proj"],
          "type": "Feature",
          "id": item_id,
          "bbox": [
            -95.62699663171666,
            45.9364285112087,
            -94.1732388249877,
            46.94756452250618
          ],
          "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                [-95.57996245196377, 45.9364285112087],
                [-95.62699663171666, 46.92355231990535],
                [-94.1853024993796, 46.94756452250618],
                [-94.1732388249877, 46.39298426493156],
                [-94.3589266638512, 45.9596401482561],
                [-94.35990594432792, 45.95746837950782],
                [-95.57996245196377, 45.9364285112087]
              ]
            ]
          },
          "properties": {
            "created": "2020-08-30T09:09:30.921Z",
            "updated": "2020-08-30T09:09:30.921Z",
            "collection": collection_id,
            "statistics": {
              "min": arr_min,
              "max": arr_max,
              "mean": arr_mean,
              "median": arr_median,
            }
          },
          "collection": collection_id,
          "links": [
            {
              "rel": "self",
              "href": f"{stac_endpoint}/stac/collections/{collection_id}/items/{item_id}"
            },
            {
              "rel": "canonical",
              "href": "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/15/S/VS/2017/11/S2A_15SVS_20171110_1_L2A/S2A_15SVS_20171110_1_L2A.json",
              "type": "application/json"
            },
            {
              "title": "Source STAC Item",
              "rel": "derived_from",
              "href": "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a/items/S2A_15SVS_20171110_1_L2A",
              "type": "application/json"
            },
            {
              "rel": "parent",
              "href": f"{stac_endpoint}/stac/collections/{collection_id}"
            },
            {
              "rel": "collection",
              "href": f"{stac_endpoint}/stac/collections/{collection_id}"
            },
            {
              "rel": "root",
              "href": f"{stac_endpoint}/stac/"
            },
            {
              "rel": "numpy",
              "href": f"{titiler_endpoint}/stac/tiles/{{z}}/{{x}}/{{y}}.npy?url={stac_endpoint}/stac/collections/{collection_id}/items/{item_id}"
            },
            {
              "title": "tms",
              "href": f"{titiler_endpoint}/stac/tiles/{{z}}/{{x}}/{{y}}.png?url=https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_13417_6725_20070828/S5_13417_6725_20070828.json&assets=B1"
            }
          ],
          "assets": {
           "B1": {
              "title": "Band 1 (coastal)",
              "type": "image/tiff; application=geotiff; profile=cloud-optimized",
              "roles": ["data"],
              "gsd": 60,
              "eo:bands": [
                {
                  "name": "B01",
                  "common_name": "coastal",
                  "center_wavelength": 0.4439,
                  "full_width_half_max": 0.027
                }
              ],
              "href": "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/14/T/QS/2018/12/S2B_14TQS_20181229_0_L2A/B01.tif",
              "proj:shape": [1830, 1830],
              "proj:transform": [
                60.0,
                0.0,
                699960.0,
                0.0,
                -60.0,
                5200020.0,
                0.0,
                0.0,
                1.0
              ]
            }
          }
        }
      ]
    }



## Request gravity statistics from Tessellata

In [ ]:
df["min"] = 0
df["max"] = 0
df["mean"] = 0
df["median"] = 0

for index, row in df.iterrows():
    body = {
        "intersects": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']]
          },
        "radius": 20000,
        "datetime": "2018-10-01T00:00:00Z/2018-11-01T12:31:12Z",
        "collections": ["fishing_pressure"],
        "statistics": ["all"]
    }
    response =  make_mocked_response()
    
    for stat in ["min", "max", "mean", "median"]:
        df[stat][index] = response["features"][0]["properties"]["statistics"][stat]

In [ ]:
df

## Plot biomass vs gravity

In [ ]:
%matplotlib inline
graph_df = df.rename(columns={'value': 'Biomass'})
ax1 = graph_df.plot.scatter(
    x='min',
    y='Biomass',
    c=graph_df.groupby('project_name').ngroup(),
    colormap='viridis',
    title='Biomass x Min Gravity',
    sharex=False
)
ax1.set_xlabel('Min Gravity', fontsize=12)
ax2 = graph_df.plot.scatter(
    x='max',
    y='Biomass',
    c=graph_df.groupby('project_name').ngroup(),
    colormap='viridis',
    title='Biomass x Max Gravity',
    sharex=False
)
ax2.set_xlabel('Max Gravity', fontsize=12)
ax3 = graph_df.plot.scatter(
    x='mean',
    y='Biomass',
    c=graph_df.groupby('project_name').ngroup(),
    colormap='magma',
    title='Biomass x Mean Gravity',
    sharex=False
)
ax3.set_xlabel('Mean Gravity', fontsize=12)
ax4 = graph_df.plot.scatter(
    x='median',
    y='Biomass',
    c=graph_df.groupby('project_name').ngroup(),
    colormap='magma',
    title='Biomass x Median Gravity',
    sharex=False
)
ax4.set_xlabel('Median Gravity', fontsize=12)

plt.show()

## View the imagery

In [ ]:
# titiler urls can be parsed from tessellata responses
titiler_url = f"{titiler_endpoint}/stac/tiles/{{z}}/{{x}}/{{y}}.png?url=https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_13417_6725_20070828/S5_13417_6725_20070828.json&assets=B1"

tile_layer = TileLayer(url=titiler_url)

m = Map(center=(67.48066409999987, -134.61833990000002), zoom=8)
m.add_layer(tile_layer);

display(m)